# NLP 핵심 요약: 이론과 실습

이 노트북은 자연어 처리(NLP)의 핵심 개념과 실습을 체계적으로 정리한 자료입니다.
앞으로의 공부에 도움이 되도록 이론과 실습 코드를 함께 제공합니다.


## 📚 목차
1. [NLP 개요](#nlp-개요)
2. [텍스트 전처리](#텍스트-전처리)
3. [특성 벡터화](#특성-벡터화)
4. [문서 군집화](#문서-군집화)
5. [문서 유사도](#문서-유사도)
6. [핵심 키워드 정리](#핵심-키워드-정리)


---

## 1. NLP 개요

### 🎯 핵심 개념

**자연어 처리(NLP)** 란 컴퓨터가 인간의 언어를 이해하고, 생성하며 활용하는 기술입니다.

### 📊 NLP 기술 적용 분야

#### 1. 자연어 이해
- **감성분석** (Sentiment Analysis): 리뷰가 긍정인가 부정인가
- **주제분류** (Topic Modeling): 뉴스 내용을 정치/스포츠/과학 등으로 분류
- **형태소 분석**: 단어의 형태소 단위로 분해
- **개체명인식** (NER): 사람이름/지역 이름 등을 구분
- **철자법 교정** (Spelling Correction)
- **스팸 탐지** (Spam Detection)

#### 2. 자연어 생성
- **문장 생성** (Text Generation)
- **번역** (Text Translation)
- **대화** (Chatbot)
- **요약** (Text Summarization)
- **이미지/비디오 설명** (Image/Video Captioning)

### 🔄 NLP 발전 단계

| **구분** | **기존 방식 (2010년대 이전)** | **최근 방식 (딥러닝 기반)** |
|----------|------------------------------|---------------------------|
| **특징** | 규칙기반, 통계기반, 전통적 ML | Deep Learning, Word Embedding |
| **접근법** | 언어학 기반 rule-based | 문맥, 의미, 어순 자동 학습 |
| **기술** | Corpus 기반 통계 모델 | Word2Vec, BERT, GPT, Transformer |

### 📝 핵심 용어

| 용어 | 설명 | 예시 |
|------|------|------|
| **말뭉치(Corpus)** | 텍스트 데이터의 모음 | 뉴스 기사 모음, 리뷰 데이터 |
| **토큰(Token)** | 텍스트의 기본 단위 | "I love NLP!" → ["I", "love", "NLP", "!"] |
| **문장(Sentence)** | 문맥의 단위 | 구두점으로 구분된 텍스트 |
| **단어(Word)** | 의미를 가지는 최소 단위 | "I", "love", "NLP" |
| **형태소(Morpheme)** | 의미의 최소 단위 | "unbreakable" → ["un-", "break", "-able"] |
| **불용어(Stopwords)** | 의미 분석에 도움 안되는 단어 | "the", "and", "is", "은", "는" |


---

## 2. 텍스트 전처리

### 🛠️ 전처리 단계

1. **텍스트 정규화** (Text Normalization)
2. **토큰화** (Tokenization)
3. **불용어 제거** (Stopword Removal)
4. **어간 추출/표제어 추출** (Stemming/Lemmatization)

### 💻 실습 코드


In [ ]:
# 필요한 라이브러리 import
import nltk
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('wordnet')

print("라이브러리 로드 완료!")


### 2.1 토큰화 (Tokenization)


In [ ]:
# 예제 텍스트
text = 'NLTK is a powerful library for NLP!!!!! I love it.'

# 단어 토큰화
word_tokens = word_tokenize(text)
print("단어 토큰화:", word_tokens)

# 문장 토큰화
sent_tokens = sent_tokenize(text)
print("문장 토큰화:", sent_tokens)

# N-gram 생성
from nltk import ngrams
bigrams = list(ngrams(word_tokens, 2))
print("Bigram:", bigrams[:5])  # 처음 5개만 출력


### 2.2 감성분석 (Sentiment Analysis)


In [ ]:
# VADER 감성분석기 초기화
analyzer = SentimentIntensityAnalyzer()

# 감성분석 예제
texts = [
    'I absolutely love this product!',
    'This is okay, I guess',
    'I hate it so much',
    'The weather is nice today'
]

for text in texts:
    scores = analyzer.polarity_scores(text)
    print(f'\n텍스트: {text}')
    print(f'점수: {scores}')
    print(f'결과: {"긍정" if scores["compound"] > 0.1 else "부정" if scores["compound"] < -0.1 else "중립"}')


### 2.3 불용어 제거 및 정규화


In [ ]:
# 텍스트 전처리 함수
def preprocess_text(text):
    """
    텍스트 전처리 함수
    1. 소문자 변환
    2. 문장 부호 제거
    3. 토큰화
    4. 불용어 제거
    5. 어근 추출
    """
    # 1. 소문자 변환
    text = text.lower()
    
    # 2. 문장 부호 제거
    punc_rem_dict = dict((ord(ch), None) for ch in string.punctuation)
    text = text.translate(punc_rem_dict)
    
    # 3. 토큰화
    tokens = word_tokenize(text)
    
    # 4. 불용어 제거
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # 5. 어근 추출
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    
    return tokens

# 전처리 테스트
sample_text = 'The Matrix is everywhere its all around us, here even in this room!!!!!'
processed = preprocess_text(sample_text)
print("원본:", sample_text)
print("전처리 후:", processed)


---

## 3. 특성 벡터화

### 📊 BOW vs Word Embedding

| **구분** | **Bag-of-Words (BOW)** | **Word Embedding** |
|----------|------------------------|-------------------|
| **개념** | 단어 출현 빈도로 문서 표현 | 단어를 실수 벡터로 표현 |
| **특징** | 순서/의미 무시, 고차원 희소 벡터 | 의미적 유사성 반영, 밀집 저차원 벡터 |
| **방법** | CountVectorizer, TF-IDF | Word2Vec, GloVe, FastText |
| **장점** | 구현 간단, 이해 쉬움 | 문맥 정보 반영, 유사 단어 근접 배치 |
| **단점** | 의미 관계/순서 정보 없음 | 많은 데이터/학습 시간 필요 |

### 💻 CountVectorizer 실습


In [ ]:
# 예제 문서들
documents = [
    'The Matrix is everywhere its all around us, here even in this room.',
    'You take the blue pill and the story ends.',
    'You take the red pill and you stay in Wonderland.'
]

# CountVectorizer 사용
count_vectorizer = CountVectorizer(
    stop_words='english',  # 불용어 제거
    ngram_range=(1, 2),    # 1-gram과 2-gram 사용
    max_features=20        # 상위 20개 특성만 사용
)

# 벡터화
count_matrix = count_vectorizer.fit_transform(documents)
print("문서-특성 행렬 형태:", count_matrix.shape)
print("\n특성 이름들:")
print(count_vectorizer.get_feature_names_out())
print("\n벡터화 결과:")
print(count_matrix.toarray())


### 📈 TF-IDF 실습

**TF-IDF 공식:**
- **TF (Term Frequency)**: tf(t,d) = 단어 t의 문서 d 내 등장 횟수 / 문서 d의 전체 단어 수
- **IDF (Inverse Document Frequency)**: idf(t) = log(N / (1 + df(t)))
- **TF-IDF**: tf-idf(t,d) = tf(t,d) × idf(t)


# TfidfVectorizer 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=20)

# TF-IDF 벡터화
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print("TF-IDF 행렬 형태:", tfidf_matrix.shape)
print("\nTF-IDF 점수:")
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=[f'문서{i+1}' for i in range(len(documents))])
print(df_tfidf.round(3))


---

## 4. 문서 군집화 & 유사도

### 🎯 K-Means 클러스터링과 코사인 유사도

**핵심 개념:**
- **K-Means**: 유사한 문서들을 K개의 그룹으로 분류
- **코사인 유사도**: 두 벡터 간 각도로 유사성 측정 (1에 가까울수록 유사)


In [ ]:
# 샘플 문서 생성
sample_docs = [
    'The GPS navigation system is very accurate and reliable.',
    'Hotel rooms are clean and comfortable with great service.',
    'The car battery life is excellent for long trips.',
    'Restaurant food quality is outstanding with fresh ingredients.',
    'The screen display is clear and bright for reading.',
    'Staff members are friendly and helpful at all times.'
]

# TF-IDF 벡터화
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.9, min_df=0.1)
doc_vecs = tfidf_vec.fit_transform(sample_docs)

# K-Means 클러스터링
kmeans = KMeans(n_clusters=3, random_state=0)
clusters = kmeans.fit_predict(doc_vecs)

# 결과 출력
doc_df = pd.DataFrame({'문서': sample_docs, '클러스터': clusters})
print("클러스터링 결과:")
print(doc_df)

# 코사인 유사도 계산
cosine_sim = cosine_similarity(doc_vecs)
print("\n코사인 유사도 행렬:")
sim_df = pd.DataFrame(cosine_sim, index=[f'문서{i+1}' for i in range(len(sample_docs))], columns=[f'문서{i+1}' for i in range(len(sample_docs))])
print(sim_df.round(3))


---

## 5. 핵심 키워드 정리

### 🔑 반드시 기억해야 할 키워드


#### **1. 기본 개념**
- **Corpus (말뭉치)**: 분석할 텍스트 데이터 모음
- **Token (토큰)**: 텍스트의 기본 분석 단위
- **Stopwords (불용어)**: 'the', 'is', 'and' 등 의미 없는 단어
- **Lemmatization (표제어 추출)**: 단어의 기본형 추출 (running → run)

#### **2. 벡터화 방법**
- **CountVectorizer**: 단어 출현 횟수 기반 벡터화
- **TF-IDF**: 단어의 중요도 가중치 계산
  - **TF** (Term Frequency): 문서 내 단어 빈도
  - **IDF** (Inverse Document Frequency): 전체 문서에서의 희소성
- **N-gram**: 연속된 N개 단어 조합 (bigram, trigram 등)

#### **3. 분석 기법**
- **Sentiment Analysis (감성분석)**: 텍스트의 긍정/부정/중립 판단
- **K-Means Clustering**: 유사 문서 그룹화
- **Cosine Similarity**: 문서 간 유사도 측정 (0~1 사이 값)


### 📚 주요 함수 치트시트


In [ ]:
# 주요 함수 사용법 정리

# 1. 토큰화
from nltk.tokenize import word_tokenize, sent_tokenize
word_tokenize("Hello world!")  # ['Hello', 'world', '!']
sent_tokenize("Hi. How are you?")  # ['Hi.', 'How are you?']

# 2. 불용어
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# 3. 표제어 추출
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('running', pos='v')  # 'run'

# 4. 감성분석
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores("I love this!")  # {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

# 5. CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english', ngram_range=(1,2), max_features=100)
# cv.fit_transform(documents)

# 6. TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.85, min_df=0.05)
# tfidf.fit_transform(documents)

# 7. K-Means
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0)
# kmeans.fit_predict(vectors)

# 8. 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(vec1, vec2)

print("주요 함수 사용법을 정리했습니다!")


### 🎯 실무 적용 팁

1. **전처리가 성능의 80%를 결정**
   - 소문자 변환, 특수문자 제거, 불용어 제거 필수
   - 도메인에 맞는 커스텀 불용어 추가

2. **TF-IDF > CountVectorizer**
   - 대부분의 경우 TF-IDF가 더 좋은 성능
   - 문서의 중요도를 잘 반영

3. **N-gram 활용**
   - unigram만 사용하면 문맥 손실
   - bigram, trigram으로 의미 조합 보존

4. **적절한 클러스터 수 선택**
   - Elbow method로 최적 K값 찾기
   - 도메인 지식 활용

5. **전처리 함수 재사용**
   - 학습 데이터와 테스트 데이터에 동일한 전처리 적용
   - 함수로 만들어 재사용성 높이기
# e3dfdfdfdfdffㄴㄴㅇㄹㅇㅇㅇㅇㅇㅇㄹtext = "Theater attendance has decreased. The THEATER industry is adapting."# 대소문자 통합
text = "Theater attendance has decreased. The THEATER industry is adapting."

transformed_text = text.lower()
transformed_text# 대소문자 통합
text = "Theater attendance has decreased. The THEATER industry is adapting."

transformed_text = text.lower()
transformed_text# 대소문자 통합
text = "Theater attendance has decreased. The THEATER industry is adapting."

transformed_text = text.lower()
transformed_text# 대소문자 통합
text = "Theater attendance has decreased. The THEATER industry is adapting."

transformed_text = text.lower()
transformed_textㅇㅇㅇ

### 📖 다음 학습 단계

#### **중급 단계**
1. **Word Embedding**: Word2Vec, GloVe, FastText
2. **한국어 NLP**: KoNLPy, 형태소 분석기 (Mecab, Okt, Komoran)
3. **실무 도구**: spaCy, Gensim
4. **Topic Modeling**: LDA, LSA

#### **고급 단계**
1. **딥러닝 NLP**: RNN, LSTM, GRU
2. **Attention & Transformer**: Self-Attention, Multi-Head Attention
3. **최신 모델**: BERT, GPT, T5
4. **Fine-tuning**: Pre-trained 모델 활용

---

## 💡 마무리

이 노트북을 통해 NLP의 핵심 개념과 실습을 체계적으로 학습했습니다.

**기억해야 할 핵심**:
- ✅ 텍스트 전처리의 중요성 (토큰화, 정규화, 불용어 제거, 표제어 추출)
- ✅ TF-IDF를 활용한 효과적인 벡터화
- ✅ K-Means를 이용한 문서 군집화
- ✅ 코사인 유사도를 통한 문서 유사성 측정

**학습 방법**:
1. 각 셀을 직접 실행해보세요
2. 다른 텍스트 데이터로 실습해보세요
3. 파라미터를 바꿔가며 결과 비교해보세요
4. 실제 프로젝트에 적용해보세요

앞으로 더 깊이 있는 NLP 공부를 위해 위의 학습 단계를 참고하세요! 🚀
